# Reformer Pretraining

## 00. Module Import

In [3]:
import os
import re
import json
import logging

import dill

from glob import glob
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from tqdm.notebook import tqdm

from reformer_pytorch import Reformer, ReformerLM
from transformers import BertTokenizer, PreTrainedTokenizer
from fairseq.optim.adafactor import Adafactor

## 01. Create WikiDataset

- Reformer를 pretrain 하기 위해 SentDataset 클래스를 만들어 준다.

In [7]:
class SentDataset(Dataset):
    
    def __init__(self, path, prefix="train"):
        
        with open(path, 'rb') as f:
            self.corpus = dill.load(f)
    
    def __len__(self):
        """Returns the number of corpus."""
        return len(self.corpus)
    
    def __getitem__(self, idx):
        return self.corpus[idx]

In [8]:
# test SentDataset 
corpus_path = '../data/corpus/kowiki_corpus.pkl'

dataset = SentDataset(corpus_path, prefix='train')

In [11]:
from pprint import pprint

print('len :', len(dataset))
print(dataset[100])
pprint(dataset[:3])

len : 4353763
수학의 기초에 대한 위기는 그 당시 수많은 논쟁에 의해 촉발되었으며, 그 논쟁에는 칸토어의 집합론과 브라우어-힐베르트 논쟁이 포함되었다.
['지미 카터 제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ '
 '1981년)이다.',
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.',
 '조지아 공과대학교를 졸업하였다.']
